In [2]:
# Install Java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Download Spark from a reliable mirror (e.g., sourceforge or other trusted mirrors)
!wget -q https://archive.apache.org/dist/spark/spark-3.4.1/spark-3.4.1-bin-hadoop3.tgz

# Extract the Spark archive
!tar xf spark-3.4.1-bin-hadoop3.tgz

# Install findspark to locate Spark on your system
!pip install -q findspark


In [3]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.4.1-bin-hadoop3"


In [4]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
file_path = "/content/drive/MyDrive/Merged_dataset.xlsx"


In [7]:
import pandas as pd

file_path = "/content/drive/MyDrive/Merged_dataset.xlsx"
pdf = pd.read_excel(file_path)
pdf.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_item_id,product_id,...,product_length_cm,product_height_cm,product_width_cm,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,seller_zip_code_prefix,seller_city,seller_state
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,1,87285b34884572647811a353c7ac498a,...,19,8,13,7c396fd4830fd04220f754e42b4e5bff,3149,sao paulo,SP,9350,maua,SP
1,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,1,87285b34884572647811a353c7ac498a,...,19,8,13,7c396fd4830fd04220f754e42b4e5bff,3149,sao paulo,SP,9350,maua,SP
2,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,1,87285b34884572647811a353c7ac498a,...,19,8,13,7c396fd4830fd04220f754e42b4e5bff,3149,sao paulo,SP,9350,maua,SP
3,128e10d95713541c87cd1a2e48201934,a20e8105f23924cd00833fd87daa0831,delivered,2017-08-15 18:29:31,2017-08-15 20:05:16,2017-08-17 15:28:33,2017-08-18 14:44:43,2017-08-28 00:00:00,1,87285b34884572647811a353c7ac498a,...,19,8,13,3a51803cc0d012c3b5dc8b7528cb05f7,3366,sao paulo,SP,9350,maua,SP
4,0e7e841ddf8f8f2de2bad69267ecfbcf,26c7ac168e1433912a51b924fbd34d34,delivered,2017-08-02 18:24:47,2017-08-02 18:43:15,2017-08-04 17:35:43,2017-08-07 18:30:01,2017-08-15 00:00:00,1,87285b34884572647811a353c7ac498a,...,19,8,13,ef0996a1a279c26e7ecbd737be23d235,2290,sao paulo,SP,9350,maua,SP


In [8]:
df = spark.createDataFrame(pdf)

In [9]:
df.show(5)

+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+-------------+--------------------+--------------------+-------------------+-----+-------------+------------------+------------+--------------------+-------------+--------------------+------------+--------------------+-----------------------+--------------------+---------------------+-------------------+--------------------------+------------------+----------------+-----------------+-----------------+----------------+--------------------+------------------------+-------------+--------------+----------------------+-----------+------------+
|            order_id|         customer_id|order_status|order_purchase_timestamp|  order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|order_item_id|          product_id|           seller_id|shipping_limit_date

In [10]:
df.createOrReplaceTempView("merged_data")
spark.sql("SELECT * FROM merged_data LIMIT 5").show()


+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+-------------+--------------------+--------------------+-------------------+-----+-------------+------------------+------------+--------------------+-------------+--------------------+------------+--------------------+-----------------------+--------------------+---------------------+-------------------+--------------------------+------------------+----------------+-----------------+-----------------+----------------+--------------------+------------------------+-------------+--------------+----------------------+-----------+------------+
|            order_id|         customer_id|order_status|order_purchase_timestamp|  order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|order_item_id|          product_id|           seller_id|shipping_limit_date

In [11]:
# Assuming df is your denormalized PySpark DataFrame
df.createOrReplaceTempView("orders")


## 1. Average delivery delay (in days)

In [12]:
spark.sql("""
SELECT
  ROUND(AVG(DATEDIFF(order_delivered_customer_date, order_estimated_delivery_date)), 2) AS avg_delivery_delay
FROM orders
WHERE order_status = 'delivered'
""").show()


+------------------+
|avg_delivery_delay|
+------------------+
|            -12.09|
+------------------+



## 2. Percentage of delivered orders that were late

In [13]:
spark.sql("""
SELECT
  ROUND(100 * SUM(CASE WHEN order_delivered_customer_date > order_estimated_delivery_date THEN 1 ELSE 0 END) / COUNT(*), 2) AS late_percentage
FROM orders
WHERE order_status = 'delivered'
""").show()

+---------------+
|late_percentage|
+---------------+
|            7.7|
+---------------+



## 3. Delay percentage by customer state

In [14]:
spark.sql("""SELECT
  customer_state,
  COUNT(*) AS total_delivered,
  SUM(CASE WHEN order_delivered_customer_date > order_estimated_delivery_date THEN 1 ELSE 0 END) AS delayed_orders,
  ROUND(100 * SUM(CASE WHEN order_delivered_customer_date > order_estimated_delivery_date THEN 1 ELSE 0 END) / COUNT(*), 2) AS delay_percentage
FROM orders
WHERE order_status = 'delivered'
GROUP BY customer_state
ORDER BY delay_percentage DESC
""").show()

+--------------+---------------+--------------+----------------+
|customer_state|total_delivered|delayed_orders|delay_percentage|
+--------------+---------------+--------------+----------------+
|            AL|            438|           105|           23.97|
|            MA|            807|           161|           19.95|
|            PI|            541|            85|           15.71|
|            SE|            382|            59|           15.45|
|            CE|           1480|           217|           14.66|
|            BA|           3845|           522|           13.58|
|            RJ|          14566|          1866|           12.81|
|            ES|           2268|           267|           11.77|
|            RN|            553|            64|           11.57|
|            TO|            329|            38|           11.55|
|            PA|           1057|           121|           11.45|
|            RR|             44|             5|           11.36|
|            PB|         

## 4. Delayed deliveries by month

In [15]:
spark.sql("""SELECT
  MONTH(order_purchase_timestamp) AS month,
  COUNT(*) AS total_orders,
  SUM(CASE WHEN order_delivered_customer_date > order_estimated_delivery_date THEN 1 ELSE 0 END) AS delayed_orders
FROM orders
WHERE order_status = 'delivered'
GROUP BY MONTH(order_purchase_timestamp)
ORDER BY month
""").show()

+-----+------------+--------------+
|month|total_orders|delayed_orders|
+-----+------------+--------------+
|    1|        9101|           541|
|    2|        9560|          1241|
|    3|       11194|          1796|
|    4|       10612|           570|
|    5|       12193|           777|
|    6|       10902|           242|
|    7|       11790|           466|
|    8|       12273|           892|
|    9|        4920|           252|
|   10|        5710|           259|
|   11|        8692|          1173|
|   12|        6262|           505|
+-----+------------+--------------+



## 5. Delay by product category

In [16]:
spark.sql("""SELECT
  product_category_name,
  COUNT(*) AS total_orders,
  SUM(CASE WHEN order_delivered_customer_date > order_estimated_delivery_date THEN 1 ELSE 0 END) AS delayed_orders,
  ROUND(100 * SUM(CASE WHEN order_delivered_customer_date > order_estimated_delivery_date THEN 1 ELSE 0 END) / COUNT(*), 2) AS delay_percentage
FROM orders
WHERE order_status = 'delivered'
GROUP BY product_category_name
ORDER BY delay_percentage DESC
""").show()

+---------------------+------------+--------------+----------------+
|product_category_name|total_orders|delayed_orders|delay_percentage|
+---------------------+------------+--------------+----------------+
|      casa_conforto_2|          28|             5|           17.86|
| moveis_colchao_e_...|          40|             5|            12.5|
|                audio|         375|            46|           12.27|
| fashion_underwear...|         139|            16|           11.51|
|      livros_tecnicos|         267|            29|           10.86|
| construcao_ferram...|         302|            32|            10.6|
|          eletronicos|        2789|           273|            9.79|
|            alimentos|         503|            49|            9.74|
|        casa_conforto|         471|            44|            9.34|
|         beleza_saude|        9747|           869|            8.92|
|                bebes|        3095|           270|            8.72|
|    moveis_escritorio|        174

## 6. Sellers with the most delayed deliveries

In [17]:
spark.sql("""SELECT
  seller_id,
  COUNT(*) AS total_orders,
  SUM(CASE WHEN order_delivered_customer_date > order_estimated_delivery_date THEN 1 ELSE 0 END) AS delayed_orders,
  ROUND(100 * SUM(CASE WHEN order_delivered_customer_date > order_estimated_delivery_date THEN 1 ELSE 0 END) / COUNT(*), 2) AS delay_percentage
FROM orders
WHERE order_status = 'delivered'
GROUP BY seller_id
ORDER BY delay_percentage DESC
LIMIT 10
""").show()

+--------------------+------------+--------------+----------------+
|           seller_id|total_orders|delayed_orders|delay_percentage|
+--------------------+------------+--------------+----------------+
|586a871d4f1221763...|           2|             2|           100.0|
|f5fea3ffed6c2e889...|           1|             1|           100.0|
|1f2eebc0e970fd3c4...|           1|             1|           100.0|
|c13ef0cfbe42f1907...|           1|             1|           100.0|
|791cfcfe22fe4a771...|           1|             1|           100.0|
|8e670472e453ba34a...|           1|             1|           100.0|
|8629a7efec1aab257...|           1|             1|           100.0|
|1b4b28463457a256e...|           1|             1|           100.0|
|19484c79cef6c062c...|           1|             1|           100.0|
|9da15f4a4ea758d9e...|           1|             1|           100.0|
+--------------------+------------+--------------+----------------+



## 7. Average approval-to-shipping delay for late vs. on-time deliveries

In [18]:
spark.sql("""SELECT
  CASE
    WHEN order_delivered_customer_date > order_estimated_delivery_date THEN 'Late'
    ELSE 'On Time'
  END AS delivery_status,
  ROUND(AVG(DATEDIFF(order_delivered_carrier_date, order_approved_at)), 2) AS avg_approval_to_ship_days
FROM orders
WHERE order_status = 'delivered'
GROUP BY delivery_status
""").show()

+---------------+-------------------------+
|delivery_status|avg_approval_to_ship_days|
+---------------+-------------------------+
|        On Time|                     2.53|
|           Late|                     5.27|
+---------------+-------------------------+



## 8. Do high-value orders get delayed more?

In [19]:
spark.sql("""SELECT
  CASE
    WHEN payment_value > 100 THEN 'High Value'
    ELSE 'Low/Medium Value'
  END AS order_value_segment,
  COUNT(*) AS total_orders,
  SUM(CASE WHEN order_delivered_customer_date > order_estimated_delivery_date THEN 1 ELSE 0 END) AS delayed_orders,
  ROUND(100 * SUM(CASE WHEN order_delivered_customer_date > order_estimated_delivery_date THEN 1 ELSE 0 END) / COUNT(*), 2) AS delay_percentage
FROM orders
WHERE order_status = 'delivered'
GROUP BY order_value_segment
""").show()

+-------------------+------------+--------------+----------------+
|order_value_segment|total_orders|delayed_orders|delay_percentage|
+-------------------+------------+--------------+----------------+
|   Low/Medium Value|       52554|          3856|            7.34|
|         High Value|       60655|          4858|            8.01|
+-------------------+------------+--------------+----------------+



## 9. Trend of delivery delays over time (monthly)

In [20]:
spark.sql("""SELECT
  DATE_FORMAT(order_purchase_timestamp, 'yyyy-MM') AS month,
  COUNT(*) AS total_orders,
  SUM(CASE WHEN order_delivered_customer_date > order_estimated_delivery_date THEN 1 ELSE 0 END) AS delayed_orders
FROM orders
WHERE order_status = 'delivered'
GROUP BY DATE_FORMAT(order_purchase_timestamp, 'yyyy-MM')
ORDER BY month
""").show()

+-------+------------+--------------+
|  month|total_orders|delayed_orders|
+-------+------------+--------------+
|2016-10|         328|             2|
|2016-12|           1|             0|
|2017-01|         955|            25|
|2017-02|        1890|            50|
|2017-03|        3010|           157|
|2017-04|        2625|           189|
|2017-05|        4174|           142|
|2017-06|        3640|           142|
|2017-07|        4666|           166|
|2017-08|        4973|           154|
|2017-09|        4920|           252|
|2017-10|        5382|           257|
|2017-11|        8692|          1173|
|2017-12|        6261|           505|
|2018-01|        8146|           516|
|2018-02|        7670|          1191|
|2018-03|        8184|          1639|
|2018-04|        7987|           381|
|2018-05|        8019|           635|
|2018-06|        7262|           100|
+-------+------------+--------------+
only showing top 20 rows



## 10. Actual vs. Estimated delivery time by state

In [21]:
spark.sql("""SELECT
  customer_state,
  ROUND(AVG(DATEDIFF(order_delivered_customer_date, order_purchase_timestamp)), 2) AS avg_actual_delivery_days,
  ROUND(AVG(DATEDIFF(order_estimated_delivery_date, order_purchase_timestamp)), 2) AS avg_estimated_delivery_days
FROM orders
WHERE order_status = 'delivered'
GROUP BY customer_state
ORDER BY avg_actual_delivery_days DESC
""").show()

+--------------+------------------------+---------------------------+
|customer_state|avg_actual_delivery_days|avg_estimated_delivery_days|
+--------------+------------------------+---------------------------+
|            RR|                    28.2|                       46.7|
|            AP|                   28.09|                      46.76|
|            AM|                   26.32|                      46.16|
|            AL|                   24.46|                      33.15|
|            PA|                   23.42|                      37.91|
|            MA|                   21.49|                      31.49|
|            SE|                   21.22|                      31.26|
|            CE|                   20.58|                      32.02|
|            AC|                   20.51|                      41.79|
|            PB|                   20.39|                      33.63|
|            RO|                   19.74|                      39.62|
|            RN|    

## 11. Product categories with highest average delivery delays + order count

In [24]:
spark.sql("""
SELECT
  product_category_name,
  COUNT(*) AS total_orders,
  ROUND(AVG(DATEDIFF(order_delivered_customer_date, order_estimated_delivery_date)), 2) AS avg_delay_days
FROM orders
WHERE order_status = 'delivered'
GROUP BY product_category_name
ORDER BY avg_delay_days DESC
""").show()

+---------------------+------------+--------------+
|product_category_name|total_orders|avg_delay_days|
+---------------------+------------+--------------+
|   artes_e_artesanato|          24|         -6.79|
| moveis_colchao_e_...|          40|         -7.18|
|      casa_conforto_2|          28|          -8.0|
| portateis_cozinha...|          14|          -9.5|
|        casa_conforto|         471|         -9.79|
|            alimentos|         503|         -9.95|
|             pc_gamer|           9|         -10.0|
|                audio|         375|        -10.18|
| fashion_underwear...|         139|        -10.73|
|    livros_importados|          59|        -11.05|
|          eletronicos|        2789|        -11.16|
| construcao_ferram...|         302|        -11.18|
|            cine_foto|          72|        -11.22|
|      fraldas_higiene|          37|        -11.35|
| fashion_roupa_fem...|          46|        -11.35|
|      livros_tecnicos|         267|        -11.39|
|           

## 12. Relation between shipping distance and delay (city mismatch as proxy for distance)

In [25]:
spark.sql("""
SELECT
  CASE
    WHEN customer_city = seller_city THEN 'Same City'
    ELSE 'Different City'
  END AS shipping_zone,
  COUNT(*) AS total_orders,
  ROUND(AVG(DATEDIFF(order_delivered_customer_date, order_estimated_delivery_date)), 2) AS avg_delay
FROM orders
WHERE order_status = 'delivered'
GROUP BY shipping_zone
""").show()


+--------------+------------+---------+
| shipping_zone|total_orders|avg_delay|
+--------------+------------+---------+
|     Same City|        5839|    -9.41|
|Different City|      107370|   -12.24|
+--------------+------------+---------+



## 13. Sellers consistently delivering late across categories

In [26]:
spark.sql("""
SELECT
  seller_id,
  product_category_name,
  COUNT(*) AS total_orders,
  SUM(CASE WHEN order_delivered_customer_date > order_estimated_delivery_date THEN 1 ELSE 0 END) AS delayed_orders,
  ROUND(100.0 * SUM(CASE WHEN order_delivered_customer_date > order_estimated_delivery_date THEN 1 ELSE 0 END) / COUNT(*), 2) AS delay_percentage
FROM orders
WHERE order_status = 'delivered'
GROUP BY seller_id, product_category_name
HAVING delayed_orders >= 10 AND delay_percentage > 50
ORDER BY delay_percentage DESC
""").show()


+--------------------+---------------------+------------+--------------+----------------+
|           seller_id|product_category_name|total_orders|delayed_orders|delay_percentage|
+--------------------+---------------------+------------+--------------+----------------+
|bbad7e518d7af88a0...|   ferramentas_jardim|          14|            10|           71.43|
|2709af9587499e95e...|         beleza_saude|          36|            21|           58.33|
+--------------------+---------------------+------------+--------------+----------------+



## 14. Correlation: freight value vs. average delay per month

In [27]:
spark.sql("""
SELECT
  DATE_FORMAT(order_purchase_timestamp, 'yyyy-MM') AS month,
  ROUND(AVG(freight_value), 2) AS avg_freight,
  ROUND(AVG(DATEDIFF(order_delivered_customer_date, order_estimated_delivery_date)), 2) AS avg_delay
FROM orders
WHERE order_status = 'delivered'
GROUP BY DATE_FORMAT(order_purchase_timestamp, 'yyyy-MM')
ORDER BY month
""").show()


+-------+-----------+---------+
|  month|avg_freight|avg_delay|
+-------+-----------+---------+
|2016-10|      19.37|   -36.27|
|2016-12|       8.72|    -22.0|
|2017-01|      17.29|   -27.99|
|2017-02|      19.99|   -19.39|
|2017-03|      19.11|   -12.44|
|2017-04|      19.32|   -13.28|
|2017-05|      19.27|   -13.45|
|2017-06|      19.49|    -12.5|
|2017-07|      19.33|   -12.66|
|2017-08|      19.06|   -13.36|
|2017-09|      19.86|   -11.37|
|2017-10|      19.74|    -12.1|
|2017-11|      19.54|    -8.39|
|2017-12|      18.98|   -13.14|
|2018-01|      19.14|   -13.13|
|2018-02|      18.63|    -8.41|
|2018-03|      20.86|    -6.97|
|2018-04|       20.4|   -13.09|
|2018-05|       19.4|   -12.38|
|2018-06|      22.42|   -19.34|
+-------+-----------+---------+
only showing top 20 rows



## 15. Percentage of late orders with low review scores (1 or 2)

In [28]:
spark.sql("""
SELECT
  COUNT(*) AS late_orders,
  SUM(CASE WHEN review_score IN (1, 2) THEN 1 ELSE 0 END) AS low_scores,
  ROUND(100.0 * SUM(CASE WHEN review_score IN (1, 2) THEN 1 ELSE 0 END) / COUNT(*), 2) AS low_score_percentage
FROM orders
WHERE order_status = 'delivered'
  AND order_delivered_customer_date > order_estimated_delivery_date
""").show()


+-----------+----------+--------------------+
|late_orders|low_scores|low_score_percentage|
+-----------+----------+--------------------+
|       8714|      4758|               54.60|
+-----------+----------+--------------------+



## 16. Longer approval time → higher chance of delay?

In [29]:
spark.sql("""
SELECT
  CASE
    WHEN DATEDIFF(order_approved_at, order_purchase_timestamp) > 1 THEN 'Slow Approval'
    ELSE 'Fast Approval'
  END AS approval_speed,
  COUNT(*) AS total_orders,
  SUM(CASE WHEN order_delivered_customer_date > order_estimated_delivery_date THEN 1 ELSE 0 END) AS delayed_orders,
  ROUND(100.0 * SUM(CASE WHEN order_delivered_customer_date > order_estimated_delivery_date THEN 1 ELSE 0 END) / COUNT(*), 2) AS delay_percentage
FROM orders
WHERE order_status = 'delivered'
GROUP BY approval_speed
""").show()


+--------------+------------+--------------+----------------+
|approval_speed|total_orders|delayed_orders|delay_percentage|
+--------------+------------+--------------+----------------+
| Fast Approval|       97802|          7233|            7.40|
| Slow Approval|       15407|          1481|            9.61|
+--------------+------------+--------------+----------------+



## 17. Delay distribution buckets (0–3, 4–7, >7 days)

In [30]:
spark.sql("""
SELECT
  CASE
    WHEN DATEDIFF(order_delivered_customer_date, order_estimated_delivery_date) BETWEEN 1 AND 3 THEN '1–3 Days Late'
    WHEN DATEDIFF(order_delivered_customer_date, order_estimated_delivery_date) BETWEEN 4 AND 7 THEN '4–7 Days Late'
    WHEN DATEDIFF(order_delivered_customer_date, order_estimated_delivery_date) > 7 THEN 'Over 7 Days Late'
  END AS delay_bucket,
  COUNT(*) AS total
FROM orders
WHERE order_status = 'delivered'
  AND order_delivered_customer_date > order_estimated_delivery_date
GROUP BY delay_bucket
""").show()


+----------------+-----+
|    delay_bucket|total|
+----------------+-----+
|            null| 1468|
|   4–7 Days Late| 1993|
|Over 7 Days Late| 3140|
|   1–3 Days Late| 2113|
+----------------+-----+



## 18. Average delay by region type (urban vs rural)

In [31]:
spark.sql("""
SELECT customer_city, COUNT(*) AS city_orders
FROM orders
GROUP BY customer_city
""").createOrReplaceTempView("city_orders")


In [32]:
spark.sql("""
SELECT
  o.customer_city,
  CASE WHEN c.city_orders > 100 THEN 'Urban' ELSE 'Rural' END AS region_type,
  ROUND(AVG(DATEDIFF(order_delivered_customer_date, order_estimated_delivery_date)), 2) AS avg_delay
FROM orders o
JOIN city_orders c ON o.customer_city = c.customer_city
WHERE o.order_status = 'delivered'
GROUP BY o.customer_city, region_type
""").show()


+--------------------+-----------+---------+
|       customer_city|region_type|avg_delay|
+--------------------+-----------+---------+
|jijoca de jericoa...|      Rural|   -23.25|
|            camacari|      Rural|    -7.49|
|                iepe|      Rural|   -11.17|
|   redencao da serra|      Rural|    -15.0|
|           igrejinha|      Rural|    -15.5|
|           arapiraca|      Rural|    -8.79|
|            barracao|      Rural|    -14.0|
|                pote|      Rural|    -15.0|
|           itaberaba|      Rural|   -12.89|
|divino das laranj...|      Rural|    -13.0|
|  aguas de sao pedro|      Rural|    -12.0|
|            itanhaem|      Rural|   -11.88|
|           boa vista|      Rural|    -18.0|
|                ijui|      Rural|   -15.49|
|             brusque|      Rural|    -11.5|
|   sao joao da barra|      Rural|    -12.0|
|           cachoeira|      Rural|    -24.8|
|              pacuja|      Rural|    -18.0|
|            buritama|      Rural|    -8.17|
|       as

##  19. Delays in high-density vs. low-density states

In [33]:
spark.sql("""
SELECT
  customer_state,
  COUNT(*) AS total_orders,
  SUM(CASE WHEN order_delivered_customer_date > order_estimated_delivery_date THEN 1 ELSE 0 END) AS delayed,
  ROUND(100.0 * SUM(CASE WHEN order_delivered_customer_date > order_estimated_delivery_date THEN 1 ELSE 0 END) / COUNT(*), 2) AS delay_percentage
FROM orders
WHERE order_status = 'delivered'
GROUP BY customer_state
ORDER BY total_orders DESC
""").show()


+--------------+------------+-------+----------------+
|customer_state|total_orders|delayed|delay_percentage|
+--------------+------------+-------+----------------+
|            SP|       47821|   2655|            5.55|
|            RJ|       14566|   1866|           12.81|
|            MG|       13219|    695|            5.26|
|            RS|        6312|    421|            6.67|
|            PR|        5790|    273|            4.72|
|            SC|        4142|    383|            9.25|
|            BA|        3845|    522|           13.58|
|            DF|        2397|    178|            7.43|
|            GO|        2305|    172|            7.46|
|            ES|        2268|    267|           11.77|
|            PE|        1796|    179|            9.97|
|            CE|        1480|    217|           14.66|
|            MT|        1060|     68|            6.42|
|            PA|        1057|    121|           11.45|
|            MS|         837|     89|           10.63|
|         

## 20. Day-of-week effect on delivery delay

In [34]:
spark.sql("""
SELECT
  DAYOFWEEK(order_purchase_timestamp) AS purchase_day,
  COUNT(*) AS total_orders,
  ROUND(AVG(DATEDIFF(order_delivered_customer_date, order_estimated_delivery_date)), 2) AS avg_delay
FROM orders
WHERE order_status = 'delivered'
GROUP BY DAYOFWEEK(order_purchase_timestamp)
ORDER BY purchase_day
""").show()


+------------+------------+---------+
|purchase_day|total_orders|avg_delay|
+------------+------------+---------+
|           1|       13447|   -12.55|
|           2|       18373|   -11.51|
|           3|       18407|   -11.98|
|           4|       17674|   -12.08|
|           5|       16908|   -12.21|
|           6|       16192|   -11.93|
|           7|       12208|   -12.68|
+------------+------------+---------+



## 21. Product categories with highest avg delay, by state

In [36]:
spark.sql("""
SELECT
    product_category_name,
    customer_state,
    ROUND(AVG(DATEDIFF(order_delivered_customer_date, order_estimated_delivery_date)), 2) AS avg_delay
FROM orders
WHERE order_delivered_customer_date > order_estimated_delivery_date
GROUP BY product_category_name, customer_state
ORDER BY avg_delay DESC
""").show()

+---------------------+--------------+---------+
|product_category_name|customer_state|avg_delay|
+---------------------+--------------+---------+
| instrumentos_musi...|            SE|    166.0|
|     moveis_decoracao|            AP|    144.0|
|        casa_conforto|            RN|    137.0|
|        esporte_lazer|            RR|    132.0|
|            alimentos|            AM|    104.0|
|   eletrodomesticos_2|            MG|     84.0|
|       consoles_games|            PA|    60.67|
| utilidades_domest...|            GO|     59.0|
|           automotivo|            SE|    51.75|
|              bebidas|            RJ|     49.0|
| moveis_colchao_e_...|            SC|     48.0|
|            alimentos|            RJ|     45.5|
|    moveis_escritorio|            RS|    44.25|
|              bebidas|            RS|     40.0|
|    moveis_escritorio|            PI|     36.4|
|     artigos_de_natal|            CE|     35.0|
|    moveis_escritorio|            MT|     33.0|
| construcao_ferram.

## 22. Correlation between number of items in order and delay

In [37]:
spark.sql("""
SELECT
    order_id,
    COUNT(product_id) AS total_items,
    DATEDIFF(order_delivered_customer_date, order_estimated_delivery_date) AS delay
FROM orders
GROUP BY order_id, order_delivered_customer_date, order_estimated_delivery_date
ORDER BY total_items DESC
""").show()

+--------------------+-----------+-----+
|            order_id|total_items|delay|
+--------------------+-----------+-----+
|895ab968e7bb0d565...|         63|  -16|
|fedcd9f7ccdc8cba3...|         38|   -5|
|ccf804e764ed5650c...|         26|   -6|
|68986e4324f6a2148...|         24|  -12|
|465c2e1bee4561cb3...|         24|   -9|
|6d58638e32674bebe...|         24|  -11|
|a3725dfe487d359b5...|         24|   -4|
|c6492b842ac190db8...|         24|   -9|
|5a3b1c29a49756e75...|         22|  -12|
|285c2e15bebd4ac83...|         22|  -13|
|1c11d0f4353b31ac3...|         21|  -17|
|8272b63d03f5f79c5...|         21|    3|
|958c6a70e60365b57...|         21|   -7|
|364f451ee38a4268d...|         20|   -8|
|1b15974a0141d54e3...|         20|   -3|
|ee9ca989fc93ba09a...|         19|   -6|
|9c54a1816e487456b...|         16|  -22|
|3a1ce4053005d669a...|         16|  -18|
|4c08f008753759d67...|         15|  -29|
|4bfcba9e084f46c8e...|         15|   -8|
+--------------------+-----------+-----+
only showing top

## 23. Repeated late deliveries vs. review score

In [38]:
spark.sql("""
SELECT
    customer_id,
    COUNT(*) AS late_order_count,
    ROUND(AVG(review_score), 2) AS avg_review_score
FROM orders
WHERE order_delivered_customer_date > order_estimated_delivery_date
GROUP BY customer_id
HAVING late_order_count > 1
ORDER BY avg_review_score ASC
""").show()

+--------------------+----------------+----------------+
|         customer_id|late_order_count|avg_review_score|
+--------------------+----------------+----------------+
|ed532487db04478df...|               2|             1.0|
|810b0fa7210f36cdb...|               2|             1.0|
|f8145d9ea34724ad4...|               2|             1.0|
|641e3183819490fc2...|               2|             1.0|
|aae53fd05ec93f09b...|               2|             1.0|
|6e598f3c0385872ad...|               2|             1.0|
|29d896af6952f87f4...|               2|             1.0|
|5d8712db3109f5c3b...|               2|             1.0|
|23090a6b2fc85e04d...|               2|             1.0|
|6e66d0911bc117a0a...|               3|             1.0|
|97a1a7d434c0aec55...|               2|             1.0|
|bd7f3abe8cf23c1f8...|               2|             1.0|
|ab0d603f0c96d5bc3...|               2|             1.0|
|96a38239347713db4...|               2|             1.0|
|14dc25a6ed75c0301...|         

## 24. Delivery delays near holidays

In [39]:
spark.sql("""
SELECT
    order_id,
    order_estimated_delivery_date,
    order_delivered_customer_date,
    DATEDIFF(order_delivered_customer_date, order_estimated_delivery_date) AS delay
FROM orders
WHERE MONTH(order_estimated_delivery_date) IN (12, 1)
AND order_delivered_customer_date > order_estimated_delivery_date
""").show()

+--------------------+-----------------------------+-----------------------------+-----+
|            order_id|order_estimated_delivery_date|order_delivered_customer_date|delay|
+--------------------+-----------------------------+-----------------------------+-----+
|1edf3cfb4e85e8094...|          2017-12-18 00:00:00|          2018-01-02 12:09:51|   15|
|9855d762d535fda66...|          2017-12-22 00:00:00|          2017-12-28 16:33:46|    6|
|9855d762d535fda66...|          2017-12-22 00:00:00|          2017-12-28 16:33:46|    6|
|8c3651ff7833e7345...|          2018-01-04 00:00:00|          2018-01-06 17:25:07|    2|
|8b79235214c377ff8...|          2017-12-12 00:00:00|          2017-12-19 18:53:11|    7|
|f5a511e584053180d...|          2017-12-13 00:00:00|          2017-12-19 19:57:04|    6|
|045797b656bf63356...|          2017-12-22 00:00:00|          2017-12-23 01:05:55|    1|
|b8b6924f004e19192...|          2017-12-18 00:00:00|          2017-12-20 20:38:46|    2|
|faa3c806c150cf6f0...

## 25. Freight value vs delay

In [40]:
spark.sql("""
SELECT
    freight_value,
    DATEDIFF(order_delivered_customer_date, order_estimated_delivery_date) AS delay
FROM orders
WHERE order_delivered_customer_date IS NOT NULL
""").show()

+-------------+-----+
|freight_value|delay|
+-------------+-----+
|         8.72|   -8|
|         8.72|   -8|
|         8.72|   -8|
|         7.78|  -10|
|         7.78|   -8|
|         14.1|   -6|
|         7.79|   -7|
|         7.79|   -3|
|        14.28|  -14|
|         7.79|  -11|
|        14.28|  -13|
|         7.78|   -7|
|         7.78|   -7|
|         7.78|  -11|
|         7.78|   -7|
|         7.78|   -7|
|        11.73|   -9|
|         15.1|  -12|
|        11.73|   -8|
|         7.78|   -8|
+-------------+-----+
only showing top 20 rows



## 26. % of early, on-time, late deliveries by state

In [41]:
spark.sql("""
SELECT
    customer_state,
    SUM(CASE WHEN order_delivered_customer_date < order_estimated_delivery_date THEN 1 ELSE 0 END) AS early,
    SUM(CASE WHEN order_delivered_customer_date = order_estimated_delivery_date THEN 1 ELSE 0 END) AS on_time,
    SUM(CASE WHEN order_delivered_customer_date > order_estimated_delivery_date THEN 1 ELSE 0 END) AS late,
    COUNT(*) AS total_orders,
    ROUND(100 * SUM(CASE WHEN order_delivered_customer_date > order_estimated_delivery_date THEN 1 ELSE 0 END) / COUNT(*), 2) AS late_pct
FROM orders
GROUP BY customer_state
ORDER BY late_pct DESC
""").show()

+--------------+-----+-------+----+------------+--------+
|customer_state|early|on_time|late|total_orders|late_pct|
+--------------+-----+-------+----+------------+--------+
|            AL|  333|      0| 105|         438|   23.97|
|            MA|  646|      0| 161|         807|   19.95|
|            PI|  456|      0|  85|         541|   15.71|
|            SE|  323|      0|  59|         382|   15.45|
|            CE| 1263|      0| 217|        1480|   14.66|
|            BA| 3323|      0| 522|        3845|   13.58|
|            RJ|12703|      0|1866|       14569|   12.81|
|            ES| 2001|      0| 267|        2268|   11.77|
|            RN|  489|      0|  64|         553|   11.57|
|            TO|  291|      0|  38|         329|   11.55|
|            PA|  936|      0| 121|        1057|   11.45|
|            RR|   39|      0|   5|          44|   11.36|
|            PB|  539|      0|  66|         605|   10.91|
|            MS|  748|      0|  89|         837|   10.63|
|            P

## 27. Peak delay periods

In [42]:
spark.sql("""
SELECT
    DATE(order_delivered_customer_date) AS delivery_date,
    COUNT(*) AS delayed_orders
FROM orders
WHERE order_delivered_customer_date > order_estimated_delivery_date
GROUP BY delivery_date
ORDER BY delayed_orders DESC
LIMIT 10
""").show()

+-------------+--------------+
|delivery_date|delayed_orders|
+-------------+--------------+
|   2018-04-03|           113|
|   2018-03-23|           105|
|   2018-03-29|           100|
|   2018-04-04|           100|
|   2018-04-10|            97|
|   2018-03-28|            96|
|   2018-04-18|            89|
|   2018-04-11|            85|
|   2018-03-27|            84|
|   2017-12-19|            83|
+-------------+--------------+



## 28. Delay impact on repeat purchases

In [43]:
spark.sql("""
SELECT
    customer_id,
    COUNT(DISTINCT order_id) AS total_orders,
    SUM(CASE WHEN order_delivered_customer_date > order_estimated_delivery_date THEN 1 ELSE 0 END) AS delayed_orders
FROM orders
GROUP BY customer_id
HAVING total_orders > 1
ORDER BY delayed_orders DESC
""").show()

+-----------+------------+--------------+
|customer_id|total_orders|delayed_orders|
+-----------+------------+--------------+
+-----------+------------+--------------+



## 29. Avg delay per seller and product category

In [44]:
spark.sql("""
SELECT
    seller_id,
    product_category_name,
    ROUND(AVG(DATEDIFF(order_delivered_customer_date, order_estimated_delivery_date)), 2) AS avg_delay
FROM orders
WHERE order_delivered_customer_date > order_estimated_delivery_date
GROUP BY seller_id, product_category_name
ORDER BY avg_delay DESC
""").show()

+--------------------+---------------------+---------+
|           seller_id|product_category_name|avg_delay|
+--------------------+---------------------+---------+
|df683dfda87bf71ac...|     moveis_decoracao|    167.0|
|c847e075301870dd1...| instrumentos_musi...|    166.0|
|e83c76265fc54bf41...|           automotivo|    159.0|
|a7f13822ceb966b07...|    moveis_escritorio|    155.0|
|4fb41dff7c5013697...|   relogios_presentes|    134.0|
|1782ce2db72c3dda6...|        esporte_lazer|    132.0|
|cac4c8e7b1ca6252d...| utilidades_domest...|    128.8|
|95e03ca3d4146e401...|     moveis_decoracao|    110.0|
|93dc87703c046b603...|   relogios_presentes|    106.0|
|2a1348e9addc1af5a...|   ferramentas_jardim|    104.0|
|056b4ada5bbc2c50c...|          eletronicos|    104.0|
|7a91bf945c6fae077...|         beleza_saude|     97.0|
|8cc6a0e5738e61a87...|            papelaria|     96.0|
|e09887ca8c7bf8a46...|     moveis_decoracao|     91.0|
|2a1348e9addc1af5a...|       consoles_games|     89.5|
|054694fa0

## 30. What is the average delay (in days) per seller and which 5 sellers have the highest cumulative delivery delays?

In [45]:
spark.sql("""
SELECT
    seller_id,
    COUNT(order_id) AS total_orders,
    ROUND(AVG(DATEDIFF(order_delivered_customer_date, order_estimated_delivery_date)), 2) AS avg_delay_days,
    ROUND(SUM(DATEDIFF(order_delivered_customer_date, order_estimated_delivery_date)), 2) AS total_delay_days
FROM orders
WHERE order_status = 'delivered'
  AND order_delivered_customer_date IS NOT NULL
  AND order_estimated_delivery_date IS NOT NULL
  AND DATEDIFF(order_delivered_customer_date, order_estimated_delivery_date) > 0
GROUP BY seller_id
ORDER BY total_delay_days DESC
LIMIT 5
""").show()


+--------------------+------------+--------------+----------------+
|           seller_id|total_orders|avg_delay_days|total_delay_days|
+--------------------+------------+--------------+----------------+
|4a3ca9315b744ce9f...|         195|         11.46|            2235|
|1f50f920176fa81da...|         160|         10.39|            1663|
|7c67e1448b00f6e96...|         123|         11.76|            1446|
|4869f7a5dfa277a7d...|         129|          9.58|            1236|
|da8622b14eb17ae28...|          97|         10.56|            1024|
+--------------------+------------+--------------+----------------+

